In [20]:
# Dependencies
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [21]:
# Pull in the charity data
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [22]:
#consider other data to drop for optimization
#potentially remove low-variance data
print(application_df['STATUS'].value_counts())
print(application_df['SPECIAL_CONSIDERATIONS'].value_counts())

1    34294
0        5
Name: STATUS, dtype: int64
N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64


In [23]:
# Drop the non-used columns and columns with low-variance data
# Name contains repeat values and could be an indicating feature
application_df = application_df.drop(['EIN'],1)
application_df = application_df.drop(['STATUS'],1)
application_df = application_df.drop(['SPECIAL_CONSIDERATIONS'],1)
application_df.head(1)

<ipython-input-23-ca8d9412331b>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  application_df = application_df.drop(['EIN'],1)
<ipython-input-23-ca8d9412331b>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  application_df = application_df.drop(['STATUS'],1)
<ipython-input-23-ca8d9412331b>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  application_df = application_df.drop(['SPECIAL_CONSIDERATIONS'],1)


,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,0,5000,1


In [24]:
# unique values per column
application_df.nunique()

NAME                19568
APPLICATION_TYPE       17
AFFILIATION             6
CLASSIFICATION         71
USE_CASE                5
ORGANIZATION            4
INCOME_AMT              9
ASK_AMT              8747
IS_SUCCESSFUL           2
dtype: int64

In [25]:
# see how many names there are
name_counts = application_df['NAME'].value_counts()
name_counts

PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: NAME, Length: 19568, dtype: int64

In [26]:
# Bin the names with occurrences 5 or fewer into "Other"
names_to_replace = list(name_counts[name_counts <= 5].index)

# Replace in dataframe
for name in names_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(name,"Other")

# Check to make sure binning was successful
print(application_df['NAME'].value_counts())

Other                                           20043
PARENT BOOSTER USA INC                           1260
TOPS CLUB INC                                     765
UNITED STATES BOWLING CONGRESS INC                700
WASHINGTON STATE UNIVERSITY                       492
                                                ...  
HABITAT FOR HUMANITY INTERNATIONAL                  6
DAMAGE PREVENTION COUNCIL OF TEXAS                  6
FLEET RESERVE ASSOCIATION                           6
HUGH OBRIAN YOUTH LEADERSHIP                        6
INTERNATIONAL CONGRESS OF CHURCHES MINISTERS        6
Name: NAME, Length: 355, dtype: int64


In [27]:
# Apply the same binning for Application type from the modeling
# Cutoff value will be 500, with counts less than that being "Other"
# use the variable name `application_types_to_replace`
applicate_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace =list(applicate_counts[applicate_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [28]:
# Apply the same binning for Classification type from the modeling
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [29]:
# insert the dummy variables into cleaned data
application_w_dum_df = pd.get_dummies(application_df)
application_w_dum_df.head(1)

,ASK_AMT,IS_SUCCESSFUL,NAME_AACE INTERNATIONAL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,NAME_ALBANY STATE UNIVERSITY NATIONAL ALUMNI ASSOCIATION,NAME_ALPHA PHI OMEGA,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [30]:
# Split our preprocessed data into our features and target arrays
X = application_w_dum_df.drop(['IS_SUCCESSFUL'], axis='columns').values
y = application_w_dum_df['IS_SUCCESSFUL'].values
# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [31]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [32]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features_count = len(X_train[0])
layer1Nodes = 90
layer2Nodes = 30
layer3Nodes = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=layer1Nodes, input_dim=input_features_count, activation='relu')
)
# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=layer2Nodes, input_dim=layer1Nodes, activation='relu')
)
# Third Hidden Layer
nn.add(
    tf.keras.layers.Dense(units=layer3Nodes, input_dim=layer2Nodes, activation='relu')
)
# Output layer
nn.add(
    tf.keras.layers.Dense(units=1, input_dim=layer3Nodes, activation='sigmoid')
)

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 90)                35640     
                                                                 
 dense_1 (Dense)             (None, 30)                2730      
                                                                 
 dense_2 (Dense)             (None, 10)                310       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 38,691
Trainable params: 38,691
Non-trainable params: 0
_________________________________________________________________


In [33]:
# Compile the model to determine loss and accuracy
nn.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])

In [34]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4758 - accuracy: 0.7726
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4333 - accuracy: 0.7896
Epoch 3/100
804/804 [==============================] - 1s 949us/step - loss: 0.4290 - accuracy: 0.7921
Epoch 4/100
804/804 [==============================] - 1s 957us/step - loss: 0.4265 - accuracy: 0.7940
Epoch 5/100
804/804 [==============================] - 1s 982us/step - loss: 0.4243 - accuracy: 0.7954
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4238 - accuracy: 0.7964
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4226 - accuracy: 0.7958
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4210 - accuracy: 0.7980
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4207 - accuracy: 0.7974
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4197 - 

804/804 [==============================] - 1s 1ms/step - loss: 0.4032 - accuracy: 0.8070
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4030 - accuracy: 0.8063
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4027 - accuracy: 0.8060
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4037 - accuracy: 0.8057
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4027 - accuracy: 0.8077
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4027 - accuracy: 0.8072
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4025 - accuracy: 0.8071
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4032 - accuracy: 0.8052
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4027 - accuracy: 0.8064
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.4026 - accuracy: 

In [37]:
#model evaluation
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss -- {model_loss}")
print(f"Accuracy -- {model_accuracy}")

268/268 - 0s - loss: 0.4623 - accuracy: 0.7904 - 135ms/epoch - 503us/step
Loss -- 0.46228864789009094
Accuracy -- 0.7904373407363892
